In [1]:
import itertools
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import datasets
import numpy as np
from datasets import load_dataset, load_metric
import torch
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers import BertForSequenceClassification
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version
from src import getTokenizedLabelDescriptions
from src import getLabelModel
from src import semsup_data_collator
from src import SemSupDataset
from src import AutoModelForMultiLabelClassification
from src import applyLightXMLNegativeSampling
from src import multilabel_metrics
from src import task_to_keys, task_to_label_keys
from src import DataTrainingArguments, ModelArguments
from src import dataset_classification_type
from src import BertForSemanticEmbedding
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
import os
from scipy.special import expit
from scipy.special import logit

In [3]:
data_files = {
    'test':  'datasets/eurlex4.3k/test.jsonl', # For GZSL
    'test_unseen' :  'datasets/eurlex4.3k/test_unseen_split1057.jsonl', # For ZSL
}

In [ ]:
raw_datasets = load_dataset(
    "json",
    data_files=data_files,
)

In [5]:
label_key = task_to_label_keys['eurlex57k'] 
label_list = [x.strip() for x in open('datasets/eurlex4.3k/all_labels.txt')]
num_labels = len(label_list)
label_list.sort() # For consistency

In [6]:
class DummyArgs: ...
model_args = DummyArgs()
model_args.model_name_or_path = 'bert-base-uncased'
model_args.negative_sampling = 'none'
model_args.config_name = None
model_args.tokenizer_name = None
model_args.label_model_name_or_path = 'prajjwal1/bert-small'
# model_args.label_model_name_or_path = 'bert-base-uncased'
model_args.cache_dir = None
model_args.use_fast_tokenizer = True
model_args.model_revision = 'main'
model_args.use_auth_token = False
model_args.semsup = True
model_args.encoder_model_type = 'bert'
model_args.arch_type = 2
model_args.devise = False
model_args.use_precomputed_embeddings = False
model_args.coil = True
model_args.token_dim = 16
data_args = DummyArgs()
data_args.task_name = 'eurlex57k'
data_args.max_seq_length = 512
data_args.pad_to_max_length = True
data_args.label_max_seq_length = 96
data_args.large_dset = False
data_args.label_description_file = 'datasets/eurlex4.3k/heir_withdescriptions_4.3k_v1_nl.json'

In [7]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    num_labels=num_labels,
    finetuning_task=data_args.task_name,
)
config.model_name_or_path = model_args.model_name_or_path
config.problem_type = dataset_classification_type[data_args.task_name]
config.negative_sampling = model_args.negative_sampling
config.semsup = model_args.semsup
config.coil = model_args.coil
config.token_dim = model_args.token_dim
if config.semsup:
    # TODO: Again take from config somehow
    config.label_hidden_size = 512
config.label_names = 'labels'
# TODO: Take from config
config.cluster_labels_dim = 64 
config.encoder_model_type = model_args.encoder_model_type
config.arch_type = model_args.arch_type
config.devise = model_args.devise
config.normalize_embeddings = False

temp_label_id = {v: i for i, v in enumerate(label_list)}
if model_args.negative_sampling == 'lightxml':
    config.group_y = [[temp_label_id[l] for l in clus] for clus in np.load('datasets/EUR-Lex/label_group_lightxml_0.npy', allow_pickle = True)]

tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    use_fast=True,
)

model = BertForSemanticEmbedding(config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Config is BertConfig {
  "_name_or_path": "bert-base-uncased",
  "arch_type": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "cluster_labels_dim": 64,
  "coil": true,
  "devise": false,
  "encoder_model_type": "bert",
  "finetuning_task": "eurlex57k",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL

In [ ]:
if model_args.semsup:
    label_model, label_tokenizer = getLabelModel(data_args, model_args)
    # tokenizedDescriptions = getTokenizedLabelDescriptions(data_args, 'datasets/EUR-Lex/all_descriptions.json', label_tokenizer)
    tokenizedDescriptions = getTokenizedLabelDescriptions(data_args, data_args.label_description_file, label_tokenizer)
    model.label_model = label_model
    model.label_tokenizer = label_tokenizer
    model.tokenizedDescriptions = tokenizedDescriptions
    
if data_args.task_name is not None:
    sentence1_key, sentence2_key = task_to_keys[data_args.task_name]

padding = "max_length"


model.config.label2id = {l: i for i, l in enumerate(label_list)}
label_to_id = model.config.label2id
model.config.id2label = {id: label for label, id in config.label2id.items()}
id2label = model.config.id2label

max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

In [10]:
def preprocess_function(examples):
    args = (
        (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
    )
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    if label_to_id is not None and label_key in examples:
        if isinstance(examples[label_key][0], list):
            labels = [[label_to_id[l] for l in examples[label_key][i]] for i in range(len(examples[label_key]))]
            result["label"] = [[1 if j in labels[i] else 0 for j in range(num_labels)] for i in range(len(labels))]
        else:
            result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]

    if model_args.negative_sampling == 'lightxml':
        result = applyLightXMLNegativeSampling(result, label_to_id, 'datasets/EUR-Lex/label_group_lightxml_0.npy', max_candidates = 200, num_labels = 3956)
    
    try: del input['labels']
    except: ...

    return result

raw_datasets = raw_datasets.with_transform(
    preprocess_function,
    # batched=True,
    # load_from_cache_file=False,
    # desc="Running tokenizer on dataset",
)

Parameter 'transform'=<function preprocess_function at 0x7f35f393fb50> of the transform datasets.arrow_dataset.Dataset.set_format couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [15]:
seen_labels = []
UNSEEN_LABELS_FILE = 'datasets/eurlex4.3k/unseen_labels_split1057.txt'
# UNSEEN_LABELS_FILE = 'datasets/eurlex4.3k/unseen_labels_split1057_fs5_median.txt'

# UNSEEN_LABELS_FILE = 'datasets/eurlex4.3k/unseen_labels_split248_root.txt'
# UNSEEN_LABELS_FILE = 'datasets/eurlex4.3k/unseen_labels.txt'

if UNSEEN_LABELS_FILE is not None:
    for line in open(UNSEEN_LABELS_FILE).readlines():
        seen_labels.append(line.strip())
else:
    seen_labels = None
if model_args.semsup:
    raw_datasets['test_unseen'] = SemSupDataset(raw_datasets['test_unseen'], data_args, data_args.label_description_file,
    label_to_id, id2label, label_tokenizer, return_desc_embeddings = True,
    seen_labels = seen_labels, add_label_name=False, max_descs_per_label = 5)
    
    raw_datasets['test'] = SemSupDataset(raw_datasets['test'], data_args, data_args.label_description_file,
    label_to_id, id2label, label_tokenizer, return_desc_embeddings = True,
    add_label_name=False, max_descs_per_label = 5)

100%|██████████| 4271/4271 [00:02<00:00, 1767.47it/s]


In [16]:
data_files

{'test': 'datasets/eurlex4.3k/test.jsonl',
 'test_unseen': 'datasets/eurlex4.3k/test_unseen_split1057.jsonl'}

In [17]:
len(seen_labels)

1057

In [18]:
data_collator = default_data_collator
torch.cuda.empty_cache()
trainer = Trainer(
    model=model,
    # args=training_args,
    train_dataset=None,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    # label_names = 'labels'
)

In [19]:
# Set to correct output dir
OUT_DIR = 'output/semsup_descs_100ep_4.3k_curie_unseen_nl1057_unseen'


# OUT_DIR = 'output/semsup_descs_100ep_4.3k_heir_unseen_coilmini_1057_nf'
# OUT_DIR = 'output/semsup_descs_100ep_4.3k_heir_unseen_split85'
checkpoint_name = 'checkpoint-10000'
# checkpoint_name = 'checkpoint-40000'


In [20]:
model.load_state_dict(torch.load(f'{OUT_DIR}/{checkpoint_name}/pytorch_model.bin'))

<All keys matched successfully>

In [ ]:
testloader = trainer.get_test_dataloader(raw_datasets['test_unseen'])

In [21]:
import json

In [22]:
all_test_texts = [json.loads(x)['text'] for x in open('datasets/eurlex4.3k/test.jsonl').readlines()]

In [ ]:
item = testloader.dataset[1004]
index = all_test_texts.index(item['text'])
labels = item['label']
knn_scores[index].nonzero(), np.array(labels).nonzero()

In [23]:
from tqdm import tqdm
import pickle
import json
def evaluate(split = 'test', NUM_SAMPLES = 1, returnDescs = False):
    ensemble_preds = None
    label_descriptions = []
    input_ids = []
    for i in range(NUM_SAMPLES):
        testloader = trainer.get_test_dataloader(raw_datasets[split])
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for item in tqdm(testloader, total = len(testloader)):
                preds = model(**{k:v.cuda() for k,v in item.items()})[1]
                all_preds.append(preds.cpu())
                all_labels.append(item['labels'].cpu())
                if returnDescs:
                    label_descriptions.append(item['desc_input_ids'].cpu())
                    input_ids.append(item['input_ids'].cpu())

                
        predictions = torch.concat(all_preds)
        labels = torch.concat(all_labels)
        if returnDescs:
            label_descriptions = torch.concat(label_descriptions)
            input_ids = torch.concat(input_ids)


        preds = predictions.cpu()
        preds = expit(preds)
        if ensemble_preds is None:
            ensemble_preds = preds.numpy()
        else:
            ensemble_preds += preds.numpy()
    if returnDescs: 
        return labels.cpu().numpy(), ensemble_preds, label_descriptions, input_ids
    else:
        return labels.cpu().numpy(), ensemble_preds

In [24]:
def get_prediction_results(preds, label_ids):
    top_values = [1,3,5, 10]
    tops = {k:0 for k in top_values}
    for i, (logit, label) in enumerate(zip(preds, label_ids)):
        logit = torch.from_numpy(logit)
        label = torch.from_numpy(label)
        _, indexes = torch.topk(logit, k = max(top_values))
        for val in top_values:
            tops[val] += len([x for x in indexes[:val] if label[x]!=0])
    precisions_at_k =  {k:v/((i+1)*k) for k,v in tops.items()}
    print('Evaluation Result: precision@{} = {}'.format(top_values, precisions_at_k))
    return precisions_at_k

## Now Finally Evaluate and store the results

In [ ]:
labels = pickle.load(open(OUT_FILE_NAME.format('labels'), 'rb'))
ensemble_preds = pickle.load(open(OUT_FILE_NAME.format('preds'), 'rb'))
precs = json.load(open(OUT_FILE_NAME.format('metrics')))

In [ ]:
knn_scores = np.load('models/eurlex/knn_test.npy')
zestlabels = [x.strip().split(';')[0] for x in open('../../zestxml/GZXML-Datasets/GZ-Eurlex-4.3K/raw/Y.txt').readlines()]
label_indices = []
for label in seen_labels:
    label_indices.append(zestlabels.index(label))
knn_scores = knn_scores[:, np.array(label_indices)]

In [51]:
knn_scores = np.load('models/eurlex/knn_test.npy')
zestlabels = [x.strip().split(';')[0] for x in open('../../zestxml/GZXML-Datasets/GZ-Eurlex-4.3K/raw/Y.txt').readlines()]
label_indices = [0 for _ in range(len(label_to_id))]
for key in label_to_id:
    label_indices[label_to_id[key]] = zestlabels.index(key)
knn_scores = knn_scores[:, np.array(label_indices)]

In [ ]:
indexes = []
for item in tqdm([json.loads(x) for x in open(data_files['test_unseen']).readlines()]):
    index = all_test_texts.index(item['text'])
    indexes.append(index)

In [ ]:
outputs_doc, logits = model.forward_input_encoder(item['input_ids'],
 item['attention_mask'], item['token_type_ids'])

In [ ]:
testloader.num_workers = 4

In [24]:
torch.cuda.empty_cache()

In [25]:
# Lets get all the label descriptions
model.eval()
testloader = trainer.get_test_dataloader(raw_datasets['test_unseen'])
# testloader = trainer.get_test_dataloader(raw_datasets['train'])
with torch.no_grad():
    tokenized_descs = testloader.dataset.class_descs_tokenized
    label_data = dict()
    for label in tqdm(tokenized_descs.keys()):
        for i in range(len(tokenized_descs[label][0])):
            desc_input_ids = torch.from_numpy(np.array(tokenized_descs[label][0][i])).unsqueeze(0).cuda()
            desc_attention_mask = torch.from_numpy(np.array(tokenized_descs[label][1][i])).unsqueeze(0).cuda()
            outputs_lab, label_embeddings, _, _ = model.forward_label_embeddings(None, None, desc_input_ids = desc_input_ids, desc_attention_mask = desc_attention_mask, return_hidden_states = True)
            lab_reps = model.tok_proj(outputs_lab.last_hidden_state @ model.label_projection.weight)
            label_data[tuple(tokenized_descs[label][0][i])] = (lab_reps, label_embeddings)

100%|██████████| 4271/4271 [01:09<00:00, 61.75it/s]


In [27]:
# Lets get all the input documents
model.eval()
testloader = trainer.get_test_dataloader(raw_datasets['test_unseen'])
# testloader = trainer.get_test_dataloader(raw_datasets['test'])
inp_data = dict()
with torch.no_grad():
    for item in tqdm(testloader.dataset):
        batch_item = dict()
        batch_item['token_type_ids'] = torch.tensor(item['token_type_ids']).unsqueeze(0).cuda()
        batch_item['input_ids'] = torch.tensor(item['input_ids']).unsqueeze(0).cuda()
        batch_item['attention_mask'] = torch.tensor(item['attention_mask']).unsqueeze(0).cuda()
        outputs_doc, logits = model.forward_input_encoder(**batch_item)
        doc_reps = model.tok_proj(outputs_doc.last_hidden_state)
        inp_data[tuple(item['input_ids'])] = (logits, doc_reps)

100%|██████████| 5337/5337 [10:36<00:00,  8.38it/s]


In [28]:
def coil_custom_eval_forward(
    self,
    input_ids: Optional[torch.Tensor] = None,
    attention_mask: Optional[torch.Tensor] = None,
    token_type_ids: Optional[torch.Tensor] = None,
    desc_input_ids = None,
    desc_attention_mask = None,
    lab_reps = None,
    label_embeddings = None
):

    outputs_doc, logits = self.forward_input_encoder(input_ids, attention_mask, token_type_ids)

    doc_reps = self.tok_proj(outputs_doc.last_hidden_state)  # D * LD * d
    # lab_reps = self.tok_proj(outputs_lab.last_hidden_state @ self.label_projection.weight)  # Q * LQ * d


    tok_scores = self.compute_tok_score_cart(
            doc_reps, input_ids,
            lab_reps, desc_input_ids.reshape(-1, desc_input_ids.shape[-1]), desc_attention_mask
    )


    logits = self.semsup_forward(logits, label_embeddings.reshape(desc_input_ids.shape[0], desc_input_ids.shape[1], -1).contiguous(), same_labels= True)
    
    new_tok_scores = torch.zeros(logits.shape, device = logits.device)
    for i in range(tok_scores.shape[1]):
        stride = tok_scores.shape[0]//tok_scores.shape[1]
        new_tok_scores[i] = tok_scores[i*stride: i*stride + stride ,i]
    # logits += new_tok_scores.contiguous()
    return logits, new_tok_scores.contiguous()

model.coil_custom_eval_forward = coil_custom_eval_forward

In [29]:
def coil_fast_eval_forward(
    self,
    input_ids: Optional[torch.Tensor] = None,
    doc_reps = None,
    logits: Optional[torch.Tensor] = None,
    desc_input_ids = None,
    desc_attention_mask = None,
    lab_reps = None,
    label_embeddings = None
):

    # outputs_doc, logits = self.forward_input_encoder(input_ids, attention_mask, token_type_ids)

    # doc_reps = self.tok_proj(outputs_doc.last_hidden_state)  # D * LD * d
    # lab_reps = self.tok_proj(outputs_lab.last_hidden_state @ self.label_projection.weight)  # Q * LQ * d


    tok_scores = self.compute_tok_score_cart(
            doc_reps, input_ids,
            lab_reps, desc_input_ids.reshape(-1, desc_input_ids.shape[-1]), desc_attention_mask
    )


    logits = self.semsup_forward(logits, label_embeddings.reshape(desc_input_ids.shape[0], desc_input_ids.shape[1], -1).contiguous(), same_labels= True)
    
    new_tok_scores = torch.zeros(logits.shape, device = logits.device)
    for i in range(tok_scores.shape[1]):
        stride = tok_scores.shape[0]//tok_scores.shape[1]
        new_tok_scores[i] = tok_scores[i*stride: i*stride + stride ,i]
    # logits += new_tok_scores.contiguous()
    return logits, new_tok_scores.contiguous()

model.coil_fast_eval_forward = coil_fast_eval_forward

In [30]:
data_files

{'test': 'datasets/eurlex4.3k/test.jsonl',
 'test_unseen': 'datasets/eurlex4.3k/test_unseen_split1057.jsonl'}

In [ ]:
# indexes = []
# for item in open(data_files['test_unseen']).readlines():
#     t = json.loads(item)['text']
#     indexes.append(all_test_texts.index(t))

In [31]:
# Now lets evaluate the model based on additional knn results
ensembled_preds = []
NUM_SAMPLES = 3
for i in range(NUM_SAMPLES):
    testloader = trainer.get_test_dataloader(raw_datasets['test_unseen'])
    # testloader = trainer.get_test_dataloader(raw_datasets['test'])
    # testloader = trainer.get_test_dataloader(raw_datasets['train'])
    all_labels = []
    all_preds = []
    all_tok_preds = []
    indexes = []
    torch.cuda.empty_cache()
    model.eval()
    from tqdm import tqdm
    with torch.no_grad():
        for item in tqdm(testloader.dataset):
            # index = all_test_texts.index(item['text'])
            # indexes.append(index)
            all_labels.append(item['label'])
            # labels = item['label']

            # Create all zeros, since we have the real positives elsewhere
            # item['labels'] = torch.tensor([[1. for x in range(len(labels))]]).cuda()
            # item['outputs_lab'], item['label_embeddings'] =  label_data[tuple(item['desc_input_ids'])]
            
            lol = [label_data[tuple(x)] for x in item['desc_input_ids']]
            item['label_embeddings'] = torch.stack([x[1] for x in lol]).squeeze().unsqueeze(0)
            item['lab_reps'] = torch.stack([x[0] for x in lol]).squeeze()

            item['desc_input_ids'] = torch.from_numpy(np.array(item['desc_input_ids'])).unsqueeze(0).cuda()
            item['desc_attention_mask'] = torch.from_numpy(np.array(item['desc_attention_mask'])).unsqueeze(0).cuda()
            # del item['text']
            del item['label']
            del item['label_desc_ids']
            del item['all_candidate_labels']
            del item['token_type_ids']
            del item['attention_mask']
            item['logits'], item['doc_reps'] = inp_data[tuple(item['input_ids'])]
            # item['token_type_ids'] = torch.tensor(item['token_type_ids']).unsqueeze(0).cuda()
            item['input_ids'] = torch.tensor(item['input_ids']).unsqueeze(0).cuda()
            # item['attention_mask'] = torch.tensor(item['attention_mask']).unsqueeze(0).cuda()
            # logits, tok_logits = model.coil_eval_forward(**item).squeeze()
            item['self'] = model
            logits, tok_logits = model.coil_fast_eval_forward(**item)
            all_preds.append(logits + tok_logits)
            all_tok_preds.append(tok_logits)
    ensembled_preds.append(all_preds)

100%|██████████| 5337/5337 [11:02<00:00,  8.05it/s]


In [ ]:
import pickle
pickle.dump(ensembled_preds, open('eurlex_nl_preds.pkl', 'wb'))
pickle.dump(all_labels     , open('eurlex_nl_alllabels.pkl', 'wb'))

In [ ]:
all_tok_preds[2].squeeze().max()

In [32]:
ep = [expit(torch.vstack(all_preds).cpu().numpy()) for all_preds in ensembled_preds]
# ep  = torch.vstack([x.squeeze() for x in all_tok_preds]).cpu().numpy()
# ep += torch.vstack([x.squeeze() for x in all_preds]).cpu().numpy()
all_labels = np.array(all_labels)

In [ ]:
get_prediction_results(ep[0],  all_labels)

In [ ]:
get_prediction_results(sum(ep) , all_labels)

In [ ]:
get_prediction_results(sum(ep) + knn_scores, all_labels)